In [ ]:
import nltk
from nltk import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
import numpy as np
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import random

In [ ]:
threshold_count = 5
df = pd.read_csv('originalDataset.csv')
dfNew = pd.DataFrame(columns = df.columns)
dfNew = df.loc[df['count'] >= threshold_count]

print(len(dfNew.loc[df['label'] == 1]))
print(len(dfNew.loc[df['label'] == 0]))

#dfNew.to_csv('dataset.csv')

In [ ]:
labeledTweets = dfNew[['content','label']]
print(labeledTweets.loc[:2])
labeledTweets = labeledTweets.sample(frac=1).reset_index(drop=True)
print(labeledTweets.loc[:2])
labeledTweets['content'] = [item.split(" ") for item in labeledTweets['content']]
print(labeledTweets.loc[:2])

In [ ]:
path = get_tmpfile("word2vec.model")
model = Word2Vec(labeledTweets['content'], size=50, min_count=1, workers=4)

In [ ]:
trainining_ratio = 0.80
training_count  = int(trainining_ratio * len(labeledTweets))
train = labeledTweets['content'].loc[:training_count - 1]
model.train(train, total_examples=training_count, epochs=20)
#model.save("word2vec.model")
#v = [np.mean(model.wv[tweet], axis=0) for tweet in m]


In [ ]:
#test = labeledTweets.loc[training_count:]
vecs = [np.mean(model.wv[x],axis=0) for x in labeledTweets['content']]
labeledTweets['vec'] = vecs

In [ ]:
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters).fit(labeledTweets['vec'].loc[:training_count - 1].values.tolist())
predictions = kmeans.predict(labeledTweets['vec'].loc[training_count:].values.tolist())
print(predictions)
print(len(predictions))

In [ ]:
greedyError = 0
for i in range(num_clusters):
    group = [labeledTweets['label'].loc[training_count+p] for p in range(len(predictions)) if predictions[p] == i]
    overallGroupSize = len(group)
    genuineCount = sum(group)
    greedyError += min(genuineCount,overallGroupSize-genuineCount)
    print("For cluster " + str(i) + ": We had overall " + str(overallGroupSize) + " points. " + str(genuineCount) + " is genuine(" + str(genuineCount/overallGroupSize)+ ")")

print("Greedy Approach yields this Accuracy: " + str(1 - (greedyError/(len(labeledTweets)-training_count))))